In [84]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [85]:
import os
import financial.data as fd
from financial.momentum.experiment.modelExperiment import ModelExperimentFactory
from financial.io.file.cache import FileCache

In [86]:
datastore = fd.CachedDataStore(path=os.environ["DATA"], cache=FileCache(cache_path=os.environ["CACHE"]+"/"))
print(datastore)                

CachedDataStore with 946 data sources [cache stats: {'size': 0, 'hit': 0, 'miss': 0, 'write': 0, 'read': 0, 'update': 0}]


In [87]:
import financial.model as fm
import financial.lab.models as labmodels
import sklearn.linear_model

class LinearScikitLearnModelFactory (labmodels.ModelFactory):
    '''
    Linear regression Scikit-Learn model factory
    '''
        
    def create_model_from_descriptors(self, 
                                      model_id: str, 
                                      hyperparameters: dict, 
                                      input_descriptor: fd.DataDescriptor, 
                                      output_descriptor: fd.DataDescriptor) -> fm.Model:
        model = sklearn.linear_model.LinearRegression()
        return fm.ScikitLearnModel(model_id, input_descriptor, output_descriptor, model, hyperparameters)
        

factory = LinearScikitLearnModelFactory()

In [88]:
start_date = "1990-01-01"
end_date = "2023-12-31"
ticker = "^GSPC"

config = {
    "mode": "global",
    "datastore": datastore,
    "ticker": ticker,
    "model_factory": factory,
    "name": "global_first_experiment",
    "start_year": start_date,
    "end_year": end_date
}

In [89]:
experiment_linear = ModelExperimentFactory.create_experiment(config)
experiment_linear.run()
print(experiment_linear.predictions)

1990-05-10   -0.002445
1990-05-11    0.000419
1990-05-14    0.000412
1990-05-15   -0.002300
1990-05-16   -0.001051
                ...   
2023-12-22   -0.003897
2023-12-26   -0.002925
2023-12-27    0.000292
2023-12-28    0.000937
2023-12-29    0.000158
Length: 8475, dtype: float64


In [ ]:
from financial.momentum.models.randomForest import RandomForestModelFactory

factory = RandomForestModelFactory()
config = {
    "mode": "global",
    "datastore": datastore,
    "ticker": ticker,
    "model_factory": factory,
    "name": "global_forest_first_experiment",
    "start_year": start_date,
    "end_year": end_date
}
experiment_forest = ModelExperimentFactory.create_experiment(config)
experiment_forest.run()

In [ ]:
print(experiment_forest.predictions)

In [ ]:
stdev = experiment_forest.target[0].stdev
mean = experiment_forest.target[0].mean
deconstructed_predictions = (experiment_forest.predictions /stdev) - mean

import pandas as pd

def reconstruct_relative(data: pd.Series, model_output: pd.Series) -> pd.Series:
    #print(model_output[:-lookahead])
    reconstructed_change = - (mean+stdev*model_output) # Sign reversal @ shift(-lookahead)
    #print("reconstructed change")
    #print(reconstructed_change[:-lookahead]) 
    # change = (final-inicial)/final => change*final = final-inicial => final = inicial/(1-change)
    reconstructed_final = data / (1-reconstructed_change)
    #print("reconstructed final")
    #print(reconstructed_final[:-lookahead].dropna()) 
    return reconstructed_final.shift(20).dropna()

data = datastore.get_data(ticker, start_date, end_date)
print(reconstruct_relative(data, deconstructed_predictions))

In [ ]:
print(data)

In [ ]:
absolute_predictions = experiment_forest.reconstruct_absolute_predictions_from_relative()
print(absolute_predictions)